# Field Dynamics Simulator (SFH Project)
This Jupyter Notebook runs the SFH Field Dynamics Simulator **locally** with interactive sliders.

You can adjust parameters (steps, entities, noise, coupling strengths) and visualize coherence & fertility dynamics.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import json
from pathlib import Path
import ipywidgets as widgets
from ipywidgets import interact

class FieldDynamicsSimulator:
    def __init__(self, n_steps=500, n_entities=200, seed=None, noise=0.05, alpha=0.1, beta=0.05):
        self.n_steps = n_steps
        self.n_entities = n_entities
        self.rng = np.random.default_rng(seed)
        self.noise = noise
        self.alpha = alpha  # fertility → coherence coupling
        self.beta = beta   # coherence → fertility coupling

        # Initialize states
        self.coherence = self.rng.random(n_entities)
        self.fertility = self.rng.random(n_entities)
        self.history = {"coherence": [], "fertility": []}

    def step(self):
        noise = self.noise * self.rng.standard_normal(self.n_entities)

        delta_c = self.alpha * (self.fertility - 0.5) - self.beta * (self.coherence - 0.5) + noise
        delta_f = self.alpha * (self.coherence - 0.5) - self.beta * (self.fertility - 0.5) + noise

        self.coherence = np.clip(self.coherence + delta_c, 0, 1)
        self.fertility = np.clip(self.fertility + delta_f, 0, 1)

        self.history["coherence"].append(self.coherence.mean())
        self.history["fertility"].append(self.fertility.mean())

    def run(self):
        for _ in range(self.n_steps):
            self.step()
        return self.history

    def plot_results(self):
        steps = np.arange(len(self.history["coherence"]))
        plt.figure(figsize=(10,6))
        plt.plot(steps, self.history["coherence"], label="Coherence", linewidth=2)
        plt.plot(steps, self.history["fertility"], label="Fertility", linewidth=2)
        plt.xlabel("Time Step")
        plt.ylabel("Average Value")
        plt.title("SFH Field Dynamics Simulation")
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()

In [ ]:
def run_interactive_sim(n_steps=500, n_entities=200, noise=0.05, alpha=0.1, beta=0.05, seed=42):
    sim = FieldDynamicsSimulator(n_steps=n_steps, n_entities=n_entities,
                                noise=noise, alpha=alpha, beta=beta, seed=seed)
    sim.run()
    sim.plot_results()

interact(run_interactive_sim,
         n_steps=widgets.IntSlider(value=500, min=100, max=2000, step=100),
         n_entities=widgets.IntSlider(value=200, min=50, max=1000, step=50),
         noise=widgets.FloatSlider(value=0.05, min=0.0, max=0.2, step=0.01),
         alpha=widgets.FloatSlider(value=0.1, min=0.0, max=0.3, step=0.01),
         beta=widgets.FloatSlider(value=0.05, min=0.0, max=0.3, step=0.01),
         seed=widgets.IntSlider(value=42, min=0, max=100))